# 1. Importar Librerías

In [1]:
import sqlite3
import pandas as pd

# 2. Trabajar con la Base de Datos

## A. Conectar con la DB

In [2]:
# Conectar a la base de datos SQLite
conexion = sqlite3.connect("sanoyfresco.db")

## B. Obtener Valores de la DB

In [3]:
# Cargar una tabla completa en un DataFrame
df = pd.read_sql_query("SELECT * FROM tickets", conexion)

# Mostrar los primeros registros del DataFrame
df.head(5)

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,1,112108,2023-02-02 00:00:00,10,4,83,49683,Pepino Kirby,0.99,4,3.96
1,1,112108,2023-02-02 00:00:00,10,4,24,13176,Bolsa de Bananas Orgánicas,2.45,4,9.80
2,1,112108,2023-02-02 00:00:00,10,4,24,47209,Aguacate Hass Orgánico,1.79,2,3.58
3,2,202279,2023-02-03 00:00:00,9,4,83,28985,Col Rizada Orgánica de Michigan,2.55,2,5.10
4,2,202279,2023-02-03 00:00:00,9,4,83,17794,Zanahorias,0.95,5,4.75


In [4]:
# Cerrar la conexión a la base de datos
conexion.close()

# 3. Información de los Datos

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id_pedido        int64  
 1   id_cliente       int64  
 2   fecha            object 
 3   hora             int64  
 4   id_departamento  int64  
 5   id_seccion       int64  
 6   id_producto      int64  
 7   nombre_producto  object 
 8   precio_unitario  float64
 9   cantidad         int64  
 10  precio_total     float64
dtypes: float64(2), int64(7), object(2)
memory usage: 417.6+ MB


La fecha no está en formato correcto

In [6]:
df["fecha"] = pd.to_datetime(df["fecha"])

# 4. Crear Nuevos Valores

In [10]:
df_cesta = df[["id_pedido", "nombre_producto"]]
df_cesta.head(5)

,id_pedido,nombre_producto
0,1,Pepino Kirby
1,1,Bolsa de Bananas Orgánicas
2,1,Aguacate Hass Orgánico
3,2,Col Rizada Orgánica de Michigan
4,2,Zanahorias


In [11]:
# Agrupar los productos por id_pedido, separarlos por ","
df_agrupado = df_cesta.groupby("id_pedido")["nombre_producto"].apply(lambda producto: ",".join(producto))
df_agrupado.head(5)

,nombre_producto
id_pedido,
1,"Pepino Kirby,Bolsa de Bananas Orgánicas,Aguaca..."
2,"Col Rizada Orgánica de Michigan,Zanahorias"
3,Espinacas Baby Orgánicas
5,"Bolsa de Bananas Orgánicas,Frambuesas Orgánica..."
10,"Banana,Cilantro Orgánico,Aguacate Orgánico,Ceb..."


In [12]:
# Aplicar pd.get_dummies() para transformar los productos en columnas con 0 o 1
df_transacciones = df_agrupado.str.get_dummies(sep=",")
df_transacciones.head(10)

,Agua con Gas de Pomelo,Aguacate Hass Orgánico,Aguacate Orgánico,Ajo Orgánico,Apio Orgánico en Ramillete Pequeño,Arándanos Orgánicos,Banana,Bolsa de Bananas Orgánicas,Calabacín Orgánico,Cebolla Amarilla Orgánica,...,Manzana Honeycrisp Orgánica,Manzanas Gala Orgánicas,Pepino Kirby,Pepino Orgánico,Racimo de Tomates Orgánicos,Rúcula Baby Orgánica,Tomates Cherry Orgánicos,Uvas Rojas sin Semillas,Zanahorias,Zanahorias Baby Orgánicas
id_pedido,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 5. Obtener Métricas

## A. Soporte

In [13]:
# Soporte para cada producto
soporte = df_transacciones.mean() * 100
soporte.sort_values(ascending=False)

,0
Banana,23.809381
Bolsa de Bananas Orgánicas,19.129710
Fresas Orgánicas,13.349170
Espinacas Baby Orgánicas,12.198935
Aguacate Hass Orgánico,10.703004
Aguacate Orgánico,8.929331
Limón Grande,7.791279
Fresas,7.241960
Limones,7.107944
Leche Entera Orgánica,6.918301


## B. Confianza

In [14]:
# Función para calcular la confianza entre dos productos en la muestra
def confianza(antecedente, consecuente):
    # Casos donde se compraron ambos productos
    conjunto_ac = df_transacciones[(df_transacciones[antecedente] == 1) & (df_transacciones[consecuente] == 1)]

    # Confianza = compras conjuntas / compras de producto A
    return len(conjunto_ac) / df_transacciones[antecedente].sum()


## C. Lift

In [15]:
# Función para calcular el lift entre dos productos en la muestra
def lift(antecedente, consecuente):
    soporte_a = df_transacciones[antecedente].mean()
    soporte_c = df_transacciones[consecuente].mean()
    conteo_ac = len(df_transacciones[(df_transacciones[antecedente] == 1) &
                                   (df_transacciones[consecuente] == 1)])
    soporte_ac = conteo_ac / len(df_transacciones)
    return soporte_ac / (soporte_a * soporte_c)

# 6. Calcular Métricas para los Productos

In [17]:
from itertools import combinations

# Definir un umbral para la confianza mínima
umbral_confianza = 0.05

asociaciones = []

# Generar combinaciones de productos y calcular confianza y lift
for antecedente, consecuente in combinations(df_transacciones.columns, 2):

    # Soporte del antecedente
    soporte_a = df_transacciones[antecedente].mean()

    # Calcular confianza
    conf = confianza(antecedente, consecuente)
    if conf > umbral_confianza:
        asociaciones.append({
            "antecedente": antecedente,
            "consecuente": consecuente,
            "soporte_a": round(soporte_a * 100,1),
            "confianza": round(conf * 100,1),
            "lift": round(lift(antecedente, consecuente),1)
        })


# Convertir las asociaciones en un DataFrame
df_asociaciones = pd.DataFrame(asociaciones)

# Ordenar las asociaciones por confianza de mayor a menor
df_asociaciones.sort_values(by = "lift", ascending = False, inplace = True)

df_asociaciones.head(10)

,antecedente,consecuente,soporte_a,confianza,lift
185,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6
224,Cilantro Orgánico,Limones,3.5,25.4,3.6
61,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5
106,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3
102,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1
85,Apio Orgánico en Ramillete Pequeño,Cebolla Amarilla Orgánica,3.4,16.2,2.8
62,Ajo Orgánico,Cebolla Roja Orgánica,5.5,9.2,2.7
89,Apio Orgánico en Ramillete Pequeño,Col Rizada Orgánica de Michigan,3.4,8.9,2.6
346,Hummus Original,Zanahorias Baby Orgánicas,3.6,9.8,2.5
349,Limones,Limón Grande,7.1,19.8,2.5


# 7. Obtener Tabla Final

## A. Valores Únicos

Generamos una tabla de los productos únicos con su 'id_producto', 'id_seccion', 'id_departamento' para enriquecer la tabla de reglas

In [18]:
# Crear una tabla con los productos únicos y las columnas correspondientes
productos_unicos = df[["id_producto", "id_seccion", "id_departamento", "nombre_producto"]].drop_duplicates()

productos_unicos

,id_producto,id_seccion,id_departamento,nombre_producto
0,49683,83,4,Pepino Kirby
1,13176,24,4,Bolsa de Bananas Orgánicas
2,47209,24,4,Aguacate Hass Orgánico
3,28985,83,4,Col Rizada Orgánica de Michigan
4,17794,83,4,Zanahorias
5,21903,123,4,Espinacas Baby Orgánicas
7,27966,123,4,Frambuesas Orgánicas
9,24852,24,4,Banana
10,31717,16,4,Cilantro Orgánico
11,47766,24,4,Aguacate Orgánico


## B. Juntar Datos

In [19]:
df_asociaciones_enriquecido = df_asociaciones.merge(productos_unicos, left_on = "antecedente", right_on = "nombre_producto", how = "left").drop(columns=["nombre_producto"])
df_asociaciones_enriquecido.columns = ["antecedente", "consecuente", "soporte_a", "confianza", "lift", "id_producto_a", "id_seccion_a", "id_departamento_a"]
df_asociaciones_enriquecido.head(10)

,antecedente,consecuente,soporte_a,confianza,lift,id_producto_a,id_seccion_a,id_departamento_a
0,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6,8518,83,4
1,Cilantro Orgánico,Limones,3.5,25.4,3.6,31717,16,4
2,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5,24964,83,4
3,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3,44359,83,4
4,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1,44359,83,4
5,Apio Orgánico en Ramillete Pequeño,Cebolla Amarilla Orgánica,3.4,16.2,2.8,44359,83,4
6,Ajo Orgánico,Cebolla Roja Orgánica,5.5,9.2,2.7,24964,83,4
7,Apio Orgánico en Ramillete Pequeño,Col Rizada Orgánica de Michigan,3.4,8.9,2.6,44359,83,4
8,Hummus Original,Zanahorias Baby Orgánicas,3.6,9.8,2.5,30489,67,20
9,Limones,Limón Grande,7.1,19.8,2.5,26209,24,4


## C. Exportar DataFrame

In [20]:
df_asociaciones_enriquecido.to_csv("reglas.csv", index = False, sep = ";", decimal = ",")